# Data Preparation

## Web-scraping via Twitter API
tweepy documentation: http://docs.tweepy.org/en/latest/

! It is really weird while scraping Trump's tweets.
Seems like being banned...
https://github.com/tweepy/tweepy/issues/1361

In [53]:
!pip install tweepy

In [11]:
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
'My Drive'


In [12]:
import os
BASE_PATH = '/gdrive/My Drive/colab_files/fp/'
DATA_PATH = '/gdrive/My Drive/colab_files/fp/dataset/'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# if not os.path.exists(os.path.join(DATA_PATH, 'harry_potter.txt')):
#     os.chdir(BASE_PATH)
#     !wget https://courses.cs.washington.edu/courses/cse599g1/19au/files/homework3.tar.gz
#     !tar -zxvf homework3.tar.gz
#     !rm homework3.tar.gz
#     !cp pt_util.py /content
#     !cp harry_potter.txt /content
# os.chdir('/content')

In [2]:
import tweepy
import time
import pandas as pd

In [57]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [58]:
politicians = ['BernieSanders', 'realDonaldTrump', 'Mike_Pence', 'JoeBiden', 'KamalaHarris']
count = 2500
tweets_df = pd.DataFrame(columns=['text', 'politician'])

for name in politicians:
    try:
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline, screen_name='@'+name, tweet_mode="extended").items(count)
        tweets_list = [{'text': tweet.full_text, 'politician': name} for tweet in tweets]
        print(len(tweets_list))
        # break
        tweets_df = tweets_df.append(tweets_list, ignore_index = True)
        # for tweet in tweets:
        #     tweets_df = tweets_df.append({'text': tweet.full_text, 'label': coding, 'politician': name}, ignore_index = True)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)

2500
20
2500
2500
2500


In [68]:
tweets = tweepy.Cursor(api.user_timeline, screen_name='@realDonaldTrump', tweet_mode="extended", count=count).items()
tweets_list = [{'text': tweet.full_text, 'politician': 'realDonaldTrump'} for tweet in tweets]
len(tweets_list)

2193

In [69]:
tweets_df = tweets_df[tweets_df['politician'] != 'realDonaldTrump']
tweets_df = tweets_df.append(tweets_list, ignore_index = True)
print(tweets_df.shape)

(12193, 2)


In [91]:
# tweets_df.to_csv(DATA_PATH+'politician_tweets.csv')
tweets_df.head()

,text,politician
0,RT @KillerMike: Going live with the OG @bernie...,BernieSanders
1,RT @ossoff: It’s been 8 months. \n\n@Perduese...,BernieSanders
2,"While the pandemic is surging, Senate Republic...",BernieSanders
3,Wages subsidized by the government during pand...,BernieSanders
4,Brandon Bernard should be alive today. We must...,BernieSanders


## Preprocessing
Process text before training

### Text Cleansing
*Ref: https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing#Lemmatization*
* Lowercasing
* Removal of URLs
* Removal of symbols (Punctuations)
* Emoji / Emoticons
* Lemmitization

In [5]:
import re
import nltk
import spacy
import string

DATA_PATH = 'drive/MyDrive/colab_files/fp/dataset/'

In [4]:
!ls drive/MyDrive/colab_files/fp/dataset

JoeBiden.csv	  politician_tweets.csv		   test.csv
KamalaHarris.csv  politician_tweets_processed.csv  train.csv
Mike_Pence.csv	  realDonaldTrump.csv		   valid.csv


In [6]:
tweets_df = pd.read_csv(DATA_PATH+'politician_tweets.csv', index_col=0)

In [7]:
# lowercasing
tweets_df["text"] = tweets_df["text"].astype(str)
tweets_df["text"] = tweets_df["text"].str.lower()
tweets_df.head(10)

,text,politician
0,rt @killermike: going live with the og @bernie...,BernieSanders
1,rt @ossoff: it’s been 8 months. \n\n@perduese...,BernieSanders
2,"while the pandemic is surging, senate republic...",BernieSanders
3,wages subsidized by the government during pand...,BernieSanders
4,brandon bernard should be alive today. we must...,BernieSanders
5,it would be outrageous and unacceptable for me...,BernieSanders
6,people all over the country are thinking: does...,BernieSanders
7,i will be voting against the defense appropria...,BernieSanders
8,corporate irresponsibility during this pandemi...,BernieSanders
9,endless money for wars? no problem. endless mo...,BernieSanders


In [8]:
# remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')
    return url_pattern.sub(r'[url]', text)
# test
text = 'https://t.co/dx3c7jne4o Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/'
remove_urls(text)

'[url] Driverless AI NLP blog post on [url]'

In [9]:
tweets_df["text"] = tweets_df["text"].apply(remove_urls)
tweets_df.head(5)

,text,politician
0,rt @killermike: going live with the og @bernie...,BernieSanders
1,rt @ossoff: it’s been 8 months. \n\n@perduese...,BernieSanders
2,"while the pandemic is surging, senate republic...",BernieSanders
3,wages subsidized by the government during pand...,BernieSanders
4,brandon bernard should be alive today. we must...,BernieSanders


In [10]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# remove_emoji("game is on 🔥🔥")

In [11]:
tweets_df["text"] = tweets_df["text"].apply(remove_emoji)
tweets_df.head(5)

,text,politician
0,rt @killermike: going live with the og @bernie...,BernieSanders
1,rt @ossoff: it’s been 8 months. \n\n@perduese...,BernieSanders
2,"while the pandemic is surging, senate republic...",BernieSanders
3,wages subsidized by the government during pand...,BernieSanders
4,brandon bernard should be alive today. we must...,BernieSanders


In [12]:
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', string.punctuation))

tweets_df["text"] = tweets_df["text"].apply(remove_punctuation)
tweets_df.head()

,text,politician
0,rt killermike going live with the og berniesan...,BernieSanders
1,rt ossoff it’s been 8 months \n\nperduesenate...,BernieSanders
2,while the pandemic is surging senate republica...,BernieSanders
3,wages subsidized by the government during pand...,BernieSanders
4,brandon bernard should be alive today we must ...,BernieSanders


In [13]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [14]:
# lemmatization
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

tweets_df["text_lemmatized"] = tweets_df["text"].apply(lemmatize_words)
tweets_df.head(5)

,text,politician,text_lemmatized
0,rt killermike going live with the og berniesan...,BernieSanders,rt killermike going live with the og berniesan...
1,rt ossoff it’s been 8 months \n\nperduesenate...,BernieSanders,rt ossoff it’s been 8 month perduesenate get o...
2,while the pandemic is surging senate republica...,BernieSanders,while the pandemic is surging senate republica...
3,wages subsidized by the government during pand...,BernieSanders,wage subsidized by the government during pande...
4,brandon bernard should be alive today we must ...,BernieSanders,brandon bernard should be alive today we must ...


In [15]:
tweets_df = tweets_df[tweets_df['text_lemmatized'] != 'url'] 

In [16]:
print(tweets_df.shape)
print(tweets_df[tweets_df['politician'] == 'realDonaldTrump'].shape)
print(tweets_df[tweets_df['politician'] == 'Mike_Pence'].shape)
print(tweets_df[tweets_df['politician'] == 'JoeBiden'].shape)
print(tweets_df[tweets_df['politician'] == 'KamalaHarris'].shape)

(11764, 3)
(1914, 3)
(2355, 3)
(2495, 3)
(2500, 3)


In [105]:
# save for record
# tweets_df.to_csv(DATA_PATH+'politician_tweets_processed.csv')

### Trim / Tokenize
https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
https://pytorch.org/text/_modules/torchtext/data/field.html
https://spacy.io/

In [30]:
import pandas as pd
import os
BASE_PATH = 'drive/MyDrive/colab_files/fp/'
DATA_PATH = 'drive/MyDrive/colab_files/fp/dataset/'
# if not os.path.exists(DATA_PATH):
#     os.makedirs(DATA_PATH)

tweets_df = pd.read_csv(DATA_PATH+'politician_tweets_processed.csv', index_col=0)

In [31]:
MAX_SIZE = 32 # avg word size of tweets is only 5.1 words
def trim_str(x):
    x = x.split(maxsplit=MAX_SIZE)
    x = ' '.join(x[:MAX_SIZE])
    return x

tweets_df["text_lemmatized"] = tweets_df["text_lemmatized"].apply(trim_str)

In [32]:
tweets_df.head(10)

,text,politician,text_lemmatized
0,rt killermike going live with the og berniesan...,BernieSanders,rt killermike going live with the og berniesan...
1,rt ossoff it’s been 8 months \n\nperduesenate...,BernieSanders,rt ossoff it’s been 8 month perduesenate get o...
2,while the pandemic is surging senate republica...,BernieSanders,while the pandemic is surging senate republica...
3,wages subsidized by the government during pand...,BernieSanders,wage subsidized by the government during pande...
4,brandon bernard should be alive today we must ...,BernieSanders,brandon bernard should be alive today we must ...
5,it would be outrageous and unacceptable for me...,BernieSanders,it would be outrageous and unacceptable for me...
6,people all over the country are thinking does ...,BernieSanders,people all over the country are thinking doe t...
7,i will be voting against the defense appropria...,BernieSanders,i will be voting against the defense appropria...
8,corporate irresponsibility during this pandemi...,BernieSanders,corporate irresponsibility during this pandemi...
9,endless money for wars no problem endless mone...,BernieSanders,endless money for war no problem endless money...


In [33]:
tweets_df["sanders"] = tweets_df["politician"].apply(lambda x: int(x == 'BernieSanders'))
tweets_df["pence"] = tweets_df["politician"].apply(lambda x: int(x == 'Mike_Pence'))
tweets_df["trump"] = tweets_df["politician"].apply(lambda x: int(x == 'realDonaldTrump'))
tweets_df["biden"] = tweets_df["politician"].apply(lambda x: int(x == 'JoeBiden'))
tweets_df["harris"] = tweets_df["politician"].apply(lambda x: int(x == 'KamalaHarris'))
tweets_df.head(5)

,text,politician,text_lemmatized,sanders,pence,trump,biden,harris
0,rt killermike going live with the og berniesan...,BernieSanders,rt killermike going live with the og berniesan...,1,0,0,0,0
1,rt ossoff it’s been 8 months \n\nperduesenate...,BernieSanders,rt ossoff it’s been 8 month perduesenate get o...,1,0,0,0,0
2,while the pandemic is surging senate republica...,BernieSanders,while the pandemic is surging senate republica...,1,0,0,0,0
3,wages subsidized by the government during pand...,BernieSanders,wage subsidized by the government during pande...,1,0,0,0,0
4,brandon bernard should be alive today we must ...,BernieSanders,brandon bernard should be alive today we must ...,1,0,0,0,0


In [1]:
# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import torch
print('Version', torch.__version__)
print('CUDA enabled:', torch.cuda.is_available())

# Preliminaries
from torchtext.data import Field, TabularDataset, BucketIterator, RawField

# Models
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Training
import torch.optim as optim

# Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import io

Version 1.7.0+cu101
CUDA enabled: True


In [2]:
USE_CUDA = True
use_cuda = USE_CUDA and torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")
print('Using device', device)

Using device cuda:0


In [36]:
# from sklearn.preprocessing import LabelBinarizer
# lb = LabelBinarizer().fit(tweets_df['politician'])
# tweets_df["label"] = tweets_df["politician"].apply(lambda x: lb.transform([x])[0])
# tweets_df.head(10)

In [37]:
# shuffle
tweets_df = tweets_df.sample(frac = 1)
train_df = tweets_df[:9000]
valid_df = tweets_df[9000:10000]
test_df = tweets_df[10000:]
valid_df.to_csv(DATA_PATH+'valid.csv')
train_df.to_csv(DATA_PATH+'train.csv')
test_df.to_csv(DATA_PATH+'test.csv')
train_df.head()

,text,politician,text_lemmatized,sanders,pence,trump,biden,harris
10898,nobody wants to report that pennsylvania and m...,realDonaldTrump,nobody want to report that pennsylvania and mi...,0,0,1,0,0
9384,thank you aapivictoryfund for having me yester...,KamalaHarris,thank you aapivictoryfund for having me yester...,0,0,0,0,1
5785,rt drbiden how it started how its g...,JoeBiden,rt drbiden how it started how it going url,0,0,0,1,0
6805,back in january when president trump signed hi...,JoeBiden,back in january when president trump signed hi...,0,0,0,1,0
5304,together were going to rebuild our economy and...,JoeBiden,together were going to rebuild our economy and...,0,0,0,1,0


In [4]:
import pandas as pd
import os
BASE_PATH = 'drive/MyDrive/colab_files/fp/'
DATA_PATH = 'drive/MyDrive/colab_files/fp/dataset/'
# prepare tensor format data
from torchtext.data import Field, TabularDataset, LabelField
# tokenizer = lambda x: x.split()
# label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float, is_target=True)
label_field = LabelField(sequential=False, use_vocab=False, batch_first=True)
text_field = Field(tokenize='spacy', include_lengths=True, batch_first=True)
fields = [('id', None), ('text', None), ('politician', None),\
          ('text_lemmatized', text_field), ('sanders', label_field),\
          ('pence', label_field), ('trump', label_field), ('biden', label_field),\
          ('harris', label_field)]

# TabularDataset
train, valid, test = TabularDataset.splits(path=DATA_PATH, train='train.csv', validation='valid.csv', test='test.csv',\
                                          format='csv', skip_header=True, fields=fields)

In [5]:
from torchtext.data import Iterator, BucketIterator
# Iterators
train_iter, valid_iter, test_iter = BucketIterator.splits((train, valid, test),\
                                                          batch_sizes=(64, 64, 64),\
                                                          device=device,\
                                                          sort_key=lambda x:len(x.text_lemmatized),\
                                                          sort_within_batch=False,\
                                                          repeat=False)
text_field.build_vocab(train, min_freq=3)

In [6]:
class BatchWrapper:
      def __init__(self, dl, x_var, y_vars):
            self.dl, self.x_var, self.y_vars = dl, x_var, y_vars # we pass in the list of attributes for x 

      def __iter__(self):
            for batch in self.dl:
                  x = getattr(batch, self.x_var) # we assume only one input in this wrapper

                  y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim=1).float()

                  yield (x, y)

      def __len__(self):
            return len(self.dl)

train_dl = BatchWrapper(train_iter, "text_lemmatized", ["sanders",	"pence", "trump", "biden", "harris"])
valid_dl = BatchWrapper(valid_iter, "text_lemmatized", ["sanders",	"pence", "trump", "biden", "harris"])
test_dl = BatchWrapper(test_iter, "text_lemmatized", ["sanders", "pence", "trump", "biden", "harris"])

# Model


## LSTM

In [26]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class LSTM(nn.Module):
    def __init__(self, dimension=128, emb_dim=512):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(len(text_field.vocab), emb_dim)
        self.dimension = dimension
        self.lstm = nn.LSTM(input_size=emb_dim,
                            hidden_size=dimension,
                            num_layers=5,
                            batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(2*dimension, 5)

    def forward(self, text, text_len):

        text_emb = self.embedding(text)

        packed_input = pack_padded_sequence(text_emb, text_len, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        out_forward = output[range(len(output)), text_len - 1, :self.dimension]
        out_reverse = output[:, 0, self.dimension:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.drop(out_reduced)

        preds = self.fc(text_fea)
        preds = torch.sigmoid(preds)
        return preds

In [22]:
# model = LSTM(embedding_dim=512, hidden_dim=128, vocab_size=len(text_field.vocab), tagset_size=5).to(device)
model = LSTM().to(device)

## Train

In [9]:
# Save and Load Functions
def save_checkpoint(save_path, model, optimizer, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model, optimizer):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [27]:
import tqdm

# Training Function
def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 5,
          eval_every = len(train_iter) // 2,
          file_path = BASE_PATH+'model',
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        for (x, len_x), y in tqdm.tqdm(train_dl, disable=True):           
            x, y = x.to(device), y.to(device)
            len_x = len_x.to('cpu')
            output = model(x, len_x)

            loss = criterion(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():                    
                  # validation loop
                    for (x, len_x), y in tqdm.tqdm(valid_dl, disable=True):
                        x, y = x.to(device), y.to(device)
                        len_x = len_x.to('cpu')
                        output = model(x, len_x)

                        loss = criterion(output, y)
                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/model.pt', model, optimizer, best_valid_loss)
                    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')


model = LSTM().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model=model, optimizer=optimizer, num_epochs=10)

Epoch [1/10], Step [70/1410], Train Loss: 0.5103, Valid Loss: 0.4468
Model saved to ==> drive/MyDrive/colab_files/fp/model/model.pt
Model saved to ==> drive/MyDrive/colab_files/fp/model/metrics.pt
Epoch [1/10], Step [140/1410], Train Loss: 0.4104, Valid Loss: 0.3846
Model saved to ==> drive/MyDrive/colab_files/fp/model/model.pt
Model saved to ==> drive/MyDrive/colab_files/fp/model/metrics.pt
Epoch [2/10], Step [210/1410], Train Loss: 0.3586, Valid Loss: 0.3445
Model saved to ==> drive/MyDrive/colab_files/fp/model/model.pt
Model saved to ==> drive/MyDrive/colab_files/fp/model/metrics.pt
Epoch [2/10], Step [280/1410], Train Loss: 0.3310, Valid Loss: 0.3165
Model saved to ==> drive/MyDrive/colab_files/fp/model/model.pt
Model saved to ==> drive/MyDrive/colab_files/fp/model/metrics.pt
Epoch [3/10], Step [350/1410], Train Loss: 0.2781, Valid Loss: 0.3025
Model saved to ==> drive/MyDrive/colab_files/fp/model/model.pt
Model saved to ==> drive/MyDrive/colab_files/fp/model/metrics.pt
Epoch [3/10

In [23]:
load_checkpoint(BASE_PATH+'model/model.pt', model, optimizer)

Model loaded from <== drive/MyDrive/colab_files/fp/model/model.pt


0.22335103433579206

In [24]:
import numpy as np
map = {
    0: 'BernieSanders',
    1: 'Mike_Pence',
    2: 'realDonaldTrump',
    3: 'JoeBiden',
    4: 'KamalaHarris'
}
politician_preds = []
numerical_preds = []
for (x, len_x), y in tqdm.tqdm(test_dl, disable=True):
    x, y = x.to(device), y.to(device)
    len_x = len_x.to('cpu')
    preds = model(x, len_x)
    preds = preds.to('cpu')
    preds = preds.data.numpy()
    for pred in preds:
        pred = 1 / (1 + np.exp(-pred))
        numerical_preds.append(pred)
        politician_preds.append(map[np.argmax(pred)])

In [28]:
test_df = pd.read_csv(DATA_PATH+'test.csv', index_col=0)
all = 0
correct = 0
for pred, label, arr in zip(politician_preds, test_df['politician'], numerical_preds):
    if pred == label:
        correct += 1
        # print(arr)
    # else:
    #     print(arr)
    all += 1
print(correct/all)

0.1989795918367347
